In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute, IBMQ
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import*
from math import*
from numpy import*
from scipy.optimize import minimize

# En caso se quiera probar con el simulador, comentar hasta el siguiente comentario
from qiskit.tools.monitor import backend_overview, job_monitor
IBMQ.load_account()
backend_overview()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_belem')

# Simulador 
#backend = Aer.get_backend('qasm_simulator')

def x_2(a,b,c):
    th = pi/2+a;
    ph1 = 0+b;
    ph2 = pi+c;
    xx = Operator([
    [exp(ph1*1.j)*cos(th), exp(ph2*1.j)*sin(th)],
    [-exp(-ph2*1.j)*sin(th), exp(-ph1*1.j)*cos(th)]
    ])
    return xx

def f(x):
    a = x[0]
    b = x[1]
    c = x[2] 
    s=[]
    shots=1000
    num=5
    n=4 #número de qubits
    
    if a<-pi/8 or a>pi/8:
        return 1e10+a+b+c
    if b<-pi/8 or b>pi/8:
        return 1e10+a+b+c
    if b<-pi/8 or c>pi/8:
        return 1e10+a+b+c
    
    for i in range(num):
        q = QuantumCircuit(n,n)
        for i in range(n):
            q.unitary(x_2(a,b,c),[i])
            q.measure(i,i)
        job = execute(q,backend,shots=shots)
        #display(job_monitor(job))
        counts = job.result().get_counts()
        
        if n==4:
            s.append(counts.get('1111',0)+counts.get('1110',0)+counts.get('1101',0)+counts.get('1100',0)
                     +counts.get('1011',0)+counts.get('1010',0)+counts.get('1001',0)+counts.get('1000',0))
            s.append(counts.get('1111',0)+counts.get('1110',0)+counts.get('1101',0)+counts.get('1100',0)
                     +counts.get('0111',0)+counts.get('0110',0)+counts.get('0101',0)+counts.get('0100',0))
            s.append(counts.get('1111',0)+counts.get('1110',0)+counts.get('1011',0)+counts.get('1010',0)
                     +counts.get('0111',0)+counts.get('0110',0)+counts.get('0011',0)+counts.get('0010',0))
            s.append(counts.get('1111',0)+counts.get('1101',0)+counts.get('1011',0)+counts.get('1001',0)
                     +counts.get('0111',0)+counts.get('0101',0)+counts.get('0011',0)+counts.get('0001',0))
        if n==3:
            s.append(counts.get('111',0)+counts.get('110',0)+counts.get('101',0)+counts.get('100',0))
            s.append(counts.get('111',0)+counts.get('110',0)+counts.get('011',0)+counts.get('010',0))
            s.append(counts.get('111',0)+counts.get('101',0)+counts.get('001',0)+counts.get('011',0))
        if n==2:
            s.append(counts.get('11',0)+counts.get('10',0))
            s.append(counts.get('11',0)+counts.get('01',0))
        if n==1:
            s.append(counts.get('1',0))
        
    promedio = sum(s)/(n*num*shots)
    st_dev = std(s)/shots
        
    return (1-promedio)*st_dev*100

x0 = [0 , 0, 0]  # Punto inicial de busqueda
NumIter = 10 # Numero de iteraciones

print("\nHallando los parámetros de ajuste :\n")
w = minimize(f, x0, method='COBYLA' , options={ 'maxiter':NumIter } ) 
print( w.message)
print( "Min COBYLA:  " + "x = %g"%w.x[0] + " , y = %g" %w.x[1] + " , z = %g" %w.x[2])
print( "Min Calculado = %g \n"%w.fun)